# Kryptografia z kluczem tajnym (symetryczna): szyfry strumieniowe
### Literatura:
1. Nowoczesna kryptografia, Aumasson
2. Kryptografia dla praktyków, Schneier

### Historia szyfrów strumieniowych
1. Zapoznaj się z działaniem szyfrem Vernama
2. Szyfr Vernama jako szyfr doskonały
![machine-vernam.jpg](attachment:machine-vernam.jpg)

#### Funkcje pomocnicze
Zadaniem poniższego zestawu funkcji jest zamiana wiadomości tekstowej (poprzez kody ASCII) do (tekstowego) ciągu 0 i 1. Chcemy analizować funkcje tak jak są one opisane w książkach.


In [ ]:
# Funkcje pomocnicze
from textwrap import wrap
import math
from codecs import encode

#tablica znaków w tablicę kodów int
def intoASCIIArray(message: str):
    int_array = []
    mesg_array = list(message)
    for i in mesg_array:
        int_array.append(ord(i))
    return int_array

#tablica kodów int w tablice znaków
def intoCharArray(message: []):
    mesg_char = []
    for i in message:
        mesg_char.append(chr(i))
    return mesg_char

# jak wyświetlić dane w postaci binarnej na n bitach
get_bin = lambda x, n: format(x, 'b').zfill(n)

#tekst ascii w postaci tablicy 8-bitowych porcji
def ASCIIToBinChunks(message_list):
    binary = []
    for x in message_list:
        binary.append(get_bin(x, 8))
    return binary

#tekst ascii w formie strumienia bitów
def ASCIIToBinStream(binary: []):
    binary_str = ""
    for x in binary:
        binary_str+=x
    return binary_str


# Przykłady

# Liczba 200 w zapisie binarnym
l = 200
bl = get_bin(l,8)
print("Liczba 200 w zapisie binarnym na 8 bitach:", bl)
bl = get_bin(l,16)
print("Liczba 200 w zapisie binarnym na 16 bitach:", bl)


messageTxt = 'The quick brown fox jumps over the lazy dog'
messageASCII = intoASCIIArray(messageTxt)
print("Tekst w formie kodów ASCII: ",messageASCII)
messageASCIIBinChunks = ASCIIToBinChunks(messageASCII)
print("Tekst w formie porcji bitów:", messageASCIIBinChunks)
messageASCIIBinStream = ASCIIToBinStream(messageASCIIBinChunks)
print("Tekst w postaci ciągu 0 i 1:", messageASCIIBinStream)
print()


#Operatory logiczne działające na napisach bitów
def XOR(bits1,bits2):
    """perform a XOR operation and return the output"""
    xor_result = ""
    for index in range(len(bits1)):
        if bits1[index] == bits2[index]:
            xor_result += '0'
        else:
            xor_result += '1'
    return xor_result

def AND(bits1,bits2):
    """perform a AND operation and return the output"""
    and_result = ""
    for index in range(len(bits1)):
        if (bits1[index] == '1') and  (bits2[index] == '1'):
            and_result += '1'
        else:
            and_result += '0'
    return and_result

def OR(bits1,bits2):
    """perform a OR operation and return the output"""
    or_result = ""
    for index in range(len(bits1)):
        if (bits1[index] == '0') and  (bits2[index] == '0'):
            or_result += '0'
        else:
            or_result += '1'
    return or_result

def NEG(bits):
    """perform a NEG operation and return the output"""
    neg_result = ""
    for index in range(len(bits)):
        if (bits[index] == '0'):
            neg_result += '1'
        else:
            neg_result += '0'
    return neg_result

print("AND", AND('0101010001101', '0101010001100'))
print("OR", OR('0101010001101', '0101010001100'))
print("XOR", XOR('0101010001101', '0101010001100'))
print("NEG", NEG('0101010001101'))

In [ ]:
# Generowanie losowych danych: funkcje i algorytmy pomocnicze
import random
random.seed("key")  #generator PRNG w python można inicjalizować tekstem

# jak wygenerować 8 losowych bitów (razy n)
def randomBytes(n):
    return bytes(random.getrandbits(8) for i in range(n))

def randomByte():
    return random.randint(0,255)

def randomByteBin():
    return get_bin(random.randint(0,255),8)

#jak wygenerować ciąg losowych bajtów
random_stream_chunks = []
for i in range(8):
    random_stream_chunks.append(randomByteBin())

print("Losowe bajty:", random_stream_chunks)

randomBinaryString = ""
for i in random_stream_chunks:
    randomBinaryString+=i

print("Strumień bitów:", randomBinaryString)

print("Pojedyncze losowe bajty:", end="")
bits1 = randomByteBin()
bits2 = get_bin(randomByte(),8)
print(bits1, bits2)

### Zadanie samodzielne dla studentów
Zaimplementuj szyfr, który utajnia strumień wiadomości jawnej przez łączenie go z pseudolosowym strumieniem


In [ ]:
s='''Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc eget augue eget sem sodales ultrices. Quisque dapibus, urna sit amet.'''
%store s >plaintext.txt
h = open("plaintext.txt", "r")

random.seed("key") #przykład


ciphertext = []
# tu wpisz swoj kod


print(ciphertext)

### Zastanów się
1. W jaki sposób łączymy strumień tekstu jawnego i strumień klucza? Wady i zalety różnych metod.

### Linear congruential generators (LCG)
Generatory liniowe kongruentne sa najprostszymi generatorami dającymi ciągi liczb o dobrych własnościach statystycznych. Kolejne wyrazy ciągu generowane sa przy pomocy formuły:

$z_{i+1}=(a\cdot z_i+c)\bmod m$

Wyraz $z_0$ nazywany jest ziarnem (_seed_). Użycie tego samego ziarna gwarantuje nam wyprodukowanie tej samej sekwencji liczb.

Charakterystyczną cechą GLK jest ich okresowość. Oczekujemy możliwie najdłuższego okresu (maksymalny to $m-1$). Najdłuższy okres otrzymamy gdy spełnione są pewne warunki (twierdzenie Hull'a-Dobell'a):

- $c$ i $m$ są względnie pierwsze,
- $a-1$ jest podzielne przez wszystkiem pierwsze czynniki $m$,
- $a-1$ jest wielokrotnoścą 4 jeśli $m$ jest wielokrotnością 4.

Przykładowe dobre wartości to $a=1103515245$, $c=12345$ dla $m=2^{31}$

Zazwyczaj generator zwraca wartość $\frac{z_i}{m}$, ale wyjście można przeskalować do dowolnej innej wartości.

Obecnie większość PRNG to tzw. _Mersenne twister_, ale ogólna idea ich użytkowania i własności jest taka sama jak w przypadku generatorów kongruentnych.

In [ ]:
def lcg(x, a, c, m):
    while True:
        x = (a * x + c) % m
        yield x

def random_uniform_sample(n, interval, seed=0):
    a, c, m = 1103515245, 12345, 2 ** 31
    bsdrand = lcg(seed, a, c, m)

    lower, upper = interval[0], interval[1]
    sample = []

    for i in range(n):
        observation = (upper - lower) * (next(bsdrand) / (2 ** 31 - 1)) + lower
        sample.append(round(observation))

    return sample

print(random_uniform_sample(100, [0,255],11))

### Zastanów sie
1. Dlaczego stosowanie generatorów PRNG jako źródłą strumienia klucza to zły pomysł?

### Kryptograficzne generatory PRNG
Urządzenie /dev/urandom stanowi podstawę dobrego generatora CPRNG

In [ ]:
import os
import struct

# random integer using os.urandom()
print(struct.unpack('i', os.urandom(4)))
# Output (258871565,)

# unsigned random integer using os.urandom()
print(struct.unpack('I', os.urandom(4)))
print(struct.unpack('I', os.urandom(4))[0] % 100)
# Output (1015967885,)

# random short number using os.urandom()
print(struct.unpack('h', os.urandom(2)))
# Output (-28882,)

# unsigned random short using os.urandom()
print(struct.unpack('H', os.urandom(2)))
# Output (29492,)

# Print random float using os.urandom()
print(struct.unpack('f', os.urandom(4)))
# Output (-4.651611836498911e+20,)

# un-singed random decimal using os.urandom()
print(struct.unpack('d', os.urandom(8)))
# Output (-1.7024488468332834e-120,)

# random char using os.urandom()
print(struct.unpack('c', os.urandom(1)))
# Output (b'\xce',)

### Zastanów się:
1. Poszukaj informacji o szyfrach binarnie addytywnych
2. Poszukaj informacji o szyfrach strumieniowych używanych w praktyce. Gdzie takie szyfry mogą być obecnie stosowane?

### Problem
Utwórz dwie różne wiadomości równej długości. Zaszyfruj je szyfrem XOR z użyciem tego samego klucza. Wyznacz alternatywę rozłączną szyfrogramów (XOR) i porównaj ją z tą samą operacją wykonaną dla tekstów jawnych.  

In [ ]:
message1 = 'secret'
message2 = 'poufny'



### Bezpieczeństwo szyfru XOR
1. Jakie znaczenie ma powyższy wynik z punktu widzenia kryptoanalizy?
2. Jeśli OTP to OK.
3. Na czym polega atak ze znanym tekstem jawnym?

### Problem

1. Utwórz dowolną wiadomość $M_1$.
2. Zaszyfruj ją swoim szyfrem XOR z kluczem $K$.
3. Wykonaj na szyfrogramie $C_1$ operację $C_2 = C_1 \oplus (111\ldots1)$.
4. Odszyfruj wiadomość $C_2$ stosując ten sam klucz $K$.
5. Porównaj wiadomości: $M_1$ i odszyfrowaną w poprzednim kroku $M_2$ (najlepiej binarnie).  